In [2]:
## imports
import csv

In [3]:
def calculateDueDate(jobs, percentage):
    sum = 0
    for job in jobs:
        sum += int(job['processing_time']);
    
    return sum * percentage;

In [4]:
def readJobsFromFile():    
    jobs = []

    spamReader = csv.reader(open('Jobs.csv', newline=''), delimiter=',', quotechar='|')
    index = 1
    for row in spamReader:
        job = {}
        job['initial_position'] = index
        job['processing_time'] = row[0]
        job['earliness_penalty'] = row[1]
        job['tardiness_penalty'] = row[2]
        jobs.append(job)
        index += 1

    return jobs

In [5]:
def objectiveFunction(result, dueDateLimit):
    completion_Time = 0 
    resultAfterFO = [] 

    for job in result:
        completion_Time= completion_Time + int(job['processing_time'])
        if(completion_Time < dueDateLimit):
            job['penalty_time'] = (int(dueDateLimit) - completion_Time) * int(job['earliness_penalty'])
            resultAfterFO.append(job)
        elif(completion_Time == dueDateLimit):
            job['penalty_time'] = (int(dueDateLimit) -completion_Time)
        elif(completion_Time > dueDateLimit):
            job['penalty_time'] = (completion_Time - int(dueDateLimit)) * int(job['tardiness_penalty'])
            resultAfterFO.append(job)
    return resultAfterFO

In [6]:
def getSumOfPenalties(resultsAfterFO):
    sumOfPenalties = 0 

    for job in resultsAfterFO:
        sumOfPenalties += int(job['penalty_time'])
        
    return sumOfPenalties;

In [7]:
def getProcessingTime(resultsAfterFO):
    processingTime = 0 

    for job in resultsAfterFO:
        processingTime += int(job['processing_time'])
        
    return processingTime;

In [8]:
def calculatePiAiBi(jobs):
    for job in jobs:
        job['pi_ai'] = int(job['processing_time']) / int(job['earliness_penalty']) 
        job['pi_bi'] = int(job['processing_time']) / int(job['tardiness_penalty']) 
    return jobs

In [9]:
def calculateJobsBeforeDueDate(sortedByPiAiDecreasing,sortedByPiBiIncreasing, dueDateLimit):
    beforeDueDate = []

    sumOfProcessingTime = 0
    for job in sortedByPiAiDecreasing:
        sumOfProcessingTime += int(job['processing_time'])
        if sumOfProcessingTime < dueDateLimit:
            beforeDueDate.append(job)
            sortedByPiBiIncreasing.remove(job)

    beforeDueDate = sorted(beforeDueDate, key=lambda k: k['pi_ai'],reverse=True )

    return beforeDueDate;

In [10]:
def merge(beforeDueDate, sortedByPiBiIncreasing):
    result = beforeDueDate + sortedByPiBiIncreasing
    return result

In [11]:
def constructiveHeuristic(jobs, percentage):
    # passo 02 - calcular o dueDate
    dueDateLimit = calculateDueDate(jobs,percentage);
    print("due date limit: ", dueDateLimit)
    # passo 03 - calcular o PiAi e o PiBi
    jobs = calculatePiAiBi(jobs)
    # passo 04 - Ordenar
    sortedByPiAiDecreasing = sorted(jobs, key=lambda k: k['pi_ai'], reverse=True )
    sortedByPiBiIncreasing = sorted(jobs, key=lambda k: k['pi_bi'] )
    # passo 05 - calcular o beforeDueDate
    beforeDueDate  = calculateJobsBeforeDueDate(sortedByPiAiDecreasing,sortedByPiBiIncreasing, dueDateLimit)
    # passo 06 - Merge
    result = merge(beforeDueDate, sortedByPiBiIncreasing)
    # passo 07 - Calcular a função objetivo
    resultsAfterFO = objectiveFunction(result, dueDateLimit)
    # passo 08 = mostrar resultados
    sumOfPenalties = getSumOfPenalties(resultsAfterFO)
    sumOfProcessingTime = getProcessingTime(resultsAfterFO)
    
    print("sum of Penalties: ", sumOfPenalties)
    print("sum of processing time: ", sumOfProcessingTime)
    print("final result: ", sumOfPenalties + sumOfProcessingTime)
    

In [12]:
jobs = readJobsFromFile()

k = 1
for i in range(0,100,10):
    print("----------------- k =", k, "--------------------------" )
    constructiveHeuristic(jobs[i:i+10], 0.2)
    k +=1
    print("--------------------------------------------------\n")
    

----------------- k = 1 --------------------------
due date limit:  23.200000000000003
sum of Penalties:  1936
sum of processing time:  116
final result:  2052
--------------------------------------------------

----------------- k = 2 --------------------------
due date limit:  25.8
sum of Penalties:  1787
sum of processing time:  129
final result:  1916
--------------------------------------------------

----------------- k = 3 --------------------------
due date limit:  25.0
sum of Penalties:  1731
sum of processing time:  119
final result:  1850
--------------------------------------------------

----------------- k = 4 --------------------------
due date limit:  20.400000000000002
sum of Penalties:  2183
sum of processing time:  102
final result:  2285
--------------------------------------------------

----------------- k = 5 --------------------------
due date limit:  18.8
sum of Penalties:  1484
sum of processing time:  94
final result:  1578
-----------------------------------